# Define spaces

In [2]:
import torch
from latentis import LatentSpace

N_SAMPLES = 100
SPACE1_DIM = 50
SPACE2_DIM = 550

space1 = LatentSpace(vectors=torch.randn(N_SAMPLES, SPACE1_DIM), name="space1")
space2 = LatentSpace(vectors=torch.randn(N_SAMPLES, SPACE2_DIM), name="space2")

# Define Translator

In [3]:
from latentis import transform
from latentis.estimate.dim_matcher import ZeroPadding
from latentis.estimate.orthogonal import SVDEstimator
from latentis.translate.translator import LatentTranslator


translator = LatentTranslator(
    random_seed=0,
    estimator=SVDEstimator(dim_matcher=ZeroPadding()),
    source_transforms=[transform.StandardScaling()],
    target_transforms=[transform.StandardScaling()],
)

# Fit Translator

In [4]:
translator.fit(source_data=space1, target_data=space2)

{'sigma_rank': 50}

# Latent Translation

In [5]:
translator(space1).shape

torch.Size([100, 550])

In [6]:
space1.shape, space2.shape

(torch.Size([100, 50]), torch.Size([100, 550]))

In [ ]:
x, y, z, subset_x, subset_y, subset_z

In [ ]:
def fit_procrustes(subset_x, subset_y):
    x = centering_x(subset_x)
    y = centering_y(subset_y)
    x, y = pad(subset_x, subset_y)

    u, s, vt = np.linalg.svd(y.T @ x)

    return u @ vt


def procrustes(x, y):
    x = fitted_centering_x(x)
    y = fitted_centering_y(y)

    return y @ x

In [ ]:
def procrustes_block(x, y):
    x = StandardScaling(x)
    y = StandardScaling(y)
    x, y = Padding(x, y)

In [ ]:
class Procrustes:

    def fit(self, x, y):
        x = standard_scaling_x(x)
        y = standard_scaling_y(y)
        x, y = pad(x, y)
        self.translation = svd(x, y)

    def transform(self, x):
        return procrustes(x, y)

    def reverse(self, y):
        y = self.translation.reverse(y, x)
        y =